In [ ]:
import os
import numpy as np
import cv2
from scipy.optimize import minimize

# Define the parameters
dt = 0.1  # Time step
initial_kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])  # Initial kernel

# Pad the kernel to match the size of the image
padding = (initial_kernel.shape[0] - 1) // 2
initial_kernel_padded = np.pad(initial_kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Objective function to minimize
def objective_function(kernel_params, training_images):
    kernel = kernel_params.reshape((3, 3))
    kernel_padded = np.pad(kernel, padding, 'constant')
    error = 0
    for image in training_images:
        denoised_image = image.copy()
        for _ in range(20):
            denoised_image = denoise_image(denoised_image, kernel_padded)
        error += np.sum((denoised_image - image) ** 2)  # Example error metric (MSE)
    return error

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u**2)
    p = dt * phi 
    return image + p

# Load the training images from a directory
training_dir = 'training_images'
training_images = []
for filename in os.listdir(training_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image = cv2.imread(os.path.join(training_dir, filename), 0)
        image = image.astype(np.float32) / 255.0
        training_images.append(image)

# Flatten initial kernel for optimization
initial_params = initial_kernel.flatten()

# Optimize the kernel parameters
optimized_params = minimize(objective_function, initial_params, args=(training_images,))

# Reshape optimized parameters to obtain the trained kernel
trained_kernel = optimized_params.x.reshape((3, 3))

# Pad the trained kernel
trained_kernel_padded = np.pad(trained_kernel, padding, 'constant')

# Load the test image
test_image = cv2.imread('test_image.jpg', 0)
test_image = test_image.astype(np.float32) / 255.0

# Denoise the test image using the trained kernel
denoised_image = test_image.copy()
for _ in range(20):
    denoised_image = denoise_image(denoised_image, trained_kernel_padded)

cv2.imwrite('denoised_image.jpg', denoised_image * 255.0)


## IMPORTANT

In [1]:
import os
import numpy as np
import cv2
from scipy.optimize import minimize
from tqdm import tqdm

# Define the parameters
dt = 0.1  # Time step
initial_kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])  # Initial kernel

# Pad the kernel to match the size of the image
padding = (initial_kernel.shape[0] - 1) // 2
initial_kernel_padded = np.pad(initial_kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Objective function to minimize
def objective_function(kernel_params, training_images):
    kernel = kernel_params.reshape((3, 3))
    kernel_padded = np.pad(kernel, padding, 'constant')
    error = 0
    for image in training_images:
        denoised_image = image.copy()
        for _ in range(20):
            denoised_image = denoise_image(denoised_image, kernel_padded)
        error += np.mean((denoised_image - image) ** 2)  # Example error metric (MSE)
    return error

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u**2)
    p = dt * phi 
    return image + p

# Load the training images from a directory
training_dir = 'training_images'
training_images = []
for filename in os.listdir(training_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image = cv2.imread(os.path.join(training_dir, filename), 0)
        image = image.astype(np.float32) / 255.0
        training_images.append(image)

# Flatten initial kernel for optimization
initial_params = initial_kernel.flatten()

# Optimize the kernel parameters with tqdm progress bar
with tqdm(total=len(training_images), desc='Training Kernel') as pbar:
    def update_progress(_):
        pbar.update(1)

    # Optimize the kernel parameters
    optimized_params = minimize(objective_function, initial_params, args=(training_images,),
                                callback=update_progress)

# Reshape optimized parameters to obtain the trained kernel
trained_kernel = optimized_params.x.reshape((3, 3))

# Pad the trained kernel
trained_kernel_padded = np.pad(trained_kernel, padding, 'constant')

# Load the test image
test_image = cv2.imread('test_image.png', 0)
test_image = test_image.astype(np.float32) / 255.0

# Denoise the test image using the trained kernel
denoised_image = test_image.copy()
for _ in range(20):
    denoised_image = denoise_image(denoised_image, trained_kernel_padded)

cv2.imwrite('denoised_image.jpg', denoised_image * 255.0)


Training Kernel:   0%|▎                                                            | 2/400 [15:17<50:43:08, 458.77s/it]


True

In [2]:
trained_kernel

array([[ 5.09229699e-09,  1.00000000e+00,  5.40467264e-10],
       [ 1.00000000e+00, -4.00000000e+00,  1.00000000e+00],
       [-2.81036184e-09,  1.00000000e+00,  1.97387560e-08]])

## Until here

In [6]:
import os
import numpy as np
import cv2
from scipy.optimize import minimize
from tqdm import tqdm

# Define the parameters
dt = 0.1  # Time step
initial_kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])  # Initial kernel

# Pad the kernel to match the size of the image
padding = (initial_kernel.shape[0] - 1) // 2
initial_kernel_padded = np.pad(initial_kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Function to compute PSNR
def compute_psnr(clean_image, denoised_image):
    mse = np.mean((clean_image - denoised_image) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Objective function to maximize PSNR
def objective_function(kernel, clean_images, noisy_images):
    kernel_padded = np.pad(kernel, padding, 'constant')
    psnr_sum = 0
    for clean_image, noisy_image in zip(clean_images, noisy_images):
        denoised_image = denoise_image(noisy_image, kernel_padded)
        psnr_sum += compute_psnr(clean_image, denoised_image)
    return -psnr_sum  # Negative to maximize PSNR

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u**2)
    p = dt * phi 
    return image + p

# Load the clean and noisy images from directories
clean_dir = 'FoETrainingSets180'
noisy_dir = 'training_images'
clean_images = []
noisy_images = []
for filename in os.listdir(clean_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        clean_image_path = os.path.join(clean_dir, filename)
        noisy_image_path = os.path.join(noisy_dir, filename)
        clean_image = cv2.imread(clean_image_path, 0)
        noisy_image = cv2.imread(noisy_image_path, 0)
        if clean_image is None or noisy_image is None:
            print(f"Failed to load image: {filename}")
            continue
        clean_image = clean_image.astype(np.float32) / 255.0
        noisy_image = noisy_image.astype(np.float32) / 255.0
        clean_images.append(clean_image)
        noisy_images.append(noisy_image)

# Optimize the kernel parameters with tqdm progress bar
with tqdm(total=500, ncols=80, desc='Optimizing Kernel') as pbar:
    def update_progress(params):
        pbar.update(1)

    # Flatten initial kernel for optimization
    initial_params = initial_kernel.flatten()

    # Optimize the kernel parameters
    optimized_params = minimize(objective_function, initial_params, args=(clean_images, noisy_images),
                            callback=update_progress, method='BFGS', options={'maxiter': 1000})


# Reshape optimized parameters to obtain the trained kernel
trained_kernel = optimized_params.x.reshape((3, 3))

# Pad the trained kernel
trained_kernel_padded = np.pad(trained_kernel, padding, 'constant')

# Load the test image
test_image = cv2.imread('test_image.png', 0)
test_image = test_image.astype(np.float32) / 255.0

# Denoise the test image using the trained kernel
denoised_image = denoise_image(test_image, trained_kernel_padded)

cv2.imwrite('denoised_image.jpg', denoised_image * 255.0)


Optimizing Kernel:   1%|▏                     | 5/500 [01:11<1:58:03, 14.31s/it]


True

## Testing Something

In [7]:
import numpy as np
import cv2
from scipy.optimize import minimize

# Define the parameters
dt = 0.1  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u ** 2)
    p = dt * phi
    return image + p

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Define the objective function to be minimized (negative PSNR)
def objective_function(kernel_flat):
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = noisy_image.copy()
    for _ in range(20):
        denoised_image = denoise_image(denoised_image, kernel)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Generate a random initial kernel
np.random.seed(42)  # Set a random seed for reproducibility
initial_kernel = np.random.rand(*kernel_size) - 0.5

# Reshape the initial kernel to a flat array for optimization
initial_kernel_flat = initial_kernel.flatten()

# Minimize the objective function to find the optimal kernel
result = minimize(objective_function, initial_kernel_flat, method='Nelder-Mead')

# Retrieve the optimal kernel and PSNR value
optimal_kernel_flat = result.x
optimal_kernel = optimal_kernel_flat.reshape(kernel_size)
optimal_psnr = -result.fun

# Print the optimal kernel and PSNR value
print("Optimal Kernel:")
print(optimal_kernel)
print("Optimal PSNR value:", optimal_psnr)

Optimal Kernel:
[[-0.4302564   0.83778959  0.41306947]
 [-0.22016006 -2.39688026  0.82945229]
 [ 0.51424396 -0.6780156   1.13199395]]
Optimal PSNR value: 21.18864314748983


In [8]:
import numpy as np
import cv2
from scipy.optimize import minimize

# Define the parameters
dt = 0.1  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u ** 2)
    p = dt * phi
    return image + p

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Define the objective function to be minimized (negative PSNR)
def objective_function(kernel_flat):
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = noisy_image.copy()
    for _ in range(20):
        denoised_image = denoise_image(denoised_image, kernel)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Generate a random initial kernel
np.random.seed(42)  # Set a random seed for reproducibility
initial_kernel = np.random.rand(*kernel_size) - 0.5

# Reshape the initial kernel to a flat array for optimization
initial_kernel_flat = initial_kernel.flatten()

# Minimize the objective function to find the optimal kernel
result = minimize(objective_function, initial_kernel_flat, method='Nelder-Mead')

# Retrieve the optimized kernel and its PSNR value
optimal_kernel_flat = result.x
optimal_kernel = optimal_kernel_flat.reshape(kernel_size)
optimal_psnr = -result.fun

# Calculate the PSNR value using the predefined kernel
predefined_kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
predefined_denoised_image = noisy_image.copy()
for _ in range(20):
    predefined_denoised_image = denoise_image(predefined_denoised_image, predefined_kernel)
predefined_psnr = compute_psnr(clean_image, predefined_denoised_image)

# Compare the PSNR values and select the most optimal
if optimal_psnr > predefined_psnr:
    most_optimal_kernel = optimal_kernel
    most_optimal_psnr = optimal_psnr
else:
    most_optimal_kernel = predefined_kernel
    most_optimal_psnr = predefined_psnr

# Print the most optimal kernel and PSNR value
print("Most Optimal Kernel:")
print(most_optimal_kernel)
print("Most Optimal PSNR value:", most_optimal_psnr)


Most Optimal Kernel:
[[-0.4302564   0.83778959  0.41306947]
 [-0.22016006 -2.39688026  0.82945229]
 [ 0.51424396 -0.6780156   1.13199395]]
Most Optimal PSNR value: 21.18864314748983


In [9]:
import numpy as np
import cv2
from scipy.optimize import minimize

# Define the parameters
dt = 0.1  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u ** 2)
    p = dt * phi
    return image + p

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Define the objective function to be minimized (negative PSNR)
def objective_function(kernel_flat):
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = noisy_image.copy()
    for _ in range(20):
        denoised_image = denoise_image(denoised_image, kernel)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Generate a random initial kernel
np.random.seed(42)  # Set a random seed for reproducibility
initial_kernel = np.random.rand(*kernel_size) - 0.5

# Reshape the initial kernel to a flat array for optimization
initial_kernel_flat = initial_kernel.flatten()

# Minimize the negative objective function to find the optimal kernel
result = minimize(objective_function, initial_kernel_flat, method='Nelder-Mead')

# Retrieve the optimized kernel and its PSNR value
optimal_kernel_flat = result.x
optimal_kernel = optimal_kernel_flat.reshape(kernel_size)
optimal_psnr = -result.fun

# Print the optimal kernel and PSNR value
print("Optimal Kernel:")
print(optimal_kernel)
print("Optimal PSNR value:", optimal_psnr)

Optimal Kernel:
[[-0.4302564   0.83778959  0.41306947]
 [-0.22016006 -2.39688026  0.82945229]
 [ 0.51424396 -0.6780156   1.13199395]]
Optimal PSNR value: 21.18864314748983


### Using GD

In [10]:
import numpy as np
import cv2

# Define the parameters
dt = 0.1  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u ** 2)
    p = dt * phi
    return image + p

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Function to compute the negative PSNR
def negative_psnr(kernel_flat):
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = noisy_image.copy()
    for _ in range(20):
        denoised_image = denoise_image(denoised_image, kernel)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Initialize the kernel randomly
np.random.seed(42)  # Set a random seed for reproducibility
initial_kernel = np.random.rand(*kernel_size) - 0.5

# Reshape the initial kernel to a flat array
initial_kernel_flat = initial_kernel.flatten()

# Define the learning rate and number of iterations
learning_rate = 0.1
num_iterations = 1000

# Perform gradient descent
current_kernel_flat = initial_kernel_flat.copy()
for i in range(num_iterations):
    gradient = np.zeros_like(current_kernel_flat)
    perturbation = np.random.normal(size=current_kernel_flat.shape)
    perturbation /= np.linalg.norm(perturbation)
    for j in range(len(current_kernel_flat)):
        perturbed_kernel_flat = current_kernel_flat + learning_rate * perturbation
        gradient[j] = (negative_psnr(perturbed_kernel_flat) - negative_psnr(current_kernel_flat)) / learning_rate
    current_kernel_flat -= learning_rate * gradient

# Reshape the final kernel
final_kernel = current_kernel_flat.reshape(kernel_size)

# Calculate the PSNR value using the final kernel
final_denoised_image = noisy_image.copy()
for _ in range(20):
    final_denoised_image = denoise_image(final_denoised_image, final_kernel)
final_psnr = compute_psnr(clean_image, final_denoised_image)

# Print the final kernel and PSNR value
print("Final Kernel:")
print(final_kernel)
print("Final PSNR value:", final_psnr)

Final Kernel:
[[-1.83392807 -1.25775388 -1.47647425]
 [-1.6098097  -2.05244955 -2.05247367]
 [-2.15038458 -1.34229204 -1.60735318]]
Final PSNR value: 7.220603049939848


### Using ADAM

In [11]:
import numpy as np
import cv2
from scipy.optimize import minimize

# Define the parameters
dt = 0.1  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u ** 2)
    p = dt * phi
    return image + p

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Function to compute the negative PSNR
def negative_psnr(kernel_flat):
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = noisy_image.copy()
    for _ in range(20):
        denoised_image = denoise_image(denoised_image, kernel)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Initialize the kernel randomly
np.random.seed(42)  # Set a random seed for reproducibility
initial_kernel = np.random.rand(*kernel_size) - 0.5

# Reshape the initial kernel to a flat array
initial_kernel_flat = initial_kernel.flatten()

# Minimize the negative PSNR using Adam optimizer
result = minimize(negative_psnr, initial_kernel_flat, method='L-BFGS-B', options={'disp': True})

# Retrieve the optimal kernel and its PSNR value
optimal_kernel_flat = result.x
optimal_kernel = optimal_kernel_flat.reshape(kernel_size)
optimal_psnr = -result.fun

# Print the optimal kernel and PSNR value
print("Optimal Kernel:")
print(optimal_kernel)
print("Optimal PSNR value:", optimal_psnr)


Optimal Kernel:
[[-0.12545988  0.45071431  0.23199394]
 [ 0.09865848 -0.34398136 -0.34400548]
 [-0.44191639  0.36617615  0.10111501]]
Optimal PSNR value: 11.280734162945249


### Adjusting Hyperparameters and 

In [13]:
import numpy as np
import cv2
from scipy.optimize import minimize

# Define the parameters
dt = 0.1  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image using a given kernel
def denoise_image(image, kernel, lambda_value):
    # Define the hyperparameter lambda
    def g(s_squared):
        return 1 / (1 + (s_squared + lambda_value**2))
    
    # Function to apply the update rule
    def update_rule(image, kernel):
        k_u = cv2.filter2D(image, -1, kernel)
        phi = k_u * g(k_u**2)
        p = dt * phi
        return image + p
    
    denoised_image = image.copy()
    for _ in range(20):
        denoised_image = update_rule(denoised_image, kernel)
    return denoised_image

# Define the objective function to be minimized (negative PSNR)
def objective_function(params):
    kernel_flat, lambda_value = params[:-1], params[-1]
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = denoise_image(noisy_image, kernel, lambda_value)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Generate a random initial set of parameters (kernel and lambda)
np.random.seed(42)  # Set a random seed for reproducibility
initial_params = np.random.rand(np.prod(kernel_size) + 1)

# Reshape the initial kernel and lambda to a flat array for optimization
initial_params_flat = initial_params.flatten()

# Minimize the negative objective function to find the optimal parameters
result = minimize(objective_function, initial_params_flat, method='Nelder-Mead')

# Retrieve the optimal parameters
optimal_params_flat = result.x
optimal_kernel_flat, optimal_lambda = optimal_params_flat[:-1], optimal_params_flat[-1]
optimal_kernel = optimal_kernel_flat.reshape(kernel_size)

# Calculate the denoised image using the optimal parameters
denoised_image = denoise_image(noisy_image, optimal_kernel, optimal_lambda)

# Calculate the PSNR value using the denoised image
optimal_psnr = compute_psnr(clean_image, denoised_image)

# Print the optimal kernel, lambda, and PSNR value
print("Optimal Kernel:")
print(optimal_kernel)
print("Optimal Lambda:", optimal_lambda)
print("Optimal PSNR value:", optimal_psnr)


Optimal Kernel:
[[ 55339.90681364 438089.41263476 230597.19084965]
 [ 34256.46548885 -73959.77310059 -33645.88257102]
 [  4466.81056124  53074.49858883 315878.5863268 ]]
Optimal Lambda: 336377.9977076431
Optimal PSNR value: 13.973660993101024


In [14]:
import numpy as np
import cv2
from scipy.optimize import minimize

# Define the parameters
dt = 0.1  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image using a given kernel
def denoise_image(image, kernel, lambda_value):
    # Define the hyperparameter lambda
    def g(s_squared):
        return 1 / (1 + (s_squared + lambda_value**2))
    
    # Function to apply the update rule
    def update_rule(image, kernel):
        k_u = cv2.filter2D(image, -1, kernel)
        phi = k_u * g(k_u**2)
        p = dt * phi
        return image + p
    
    denoised_image = image.copy()
    for _ in range(20):
        denoised_image = update_rule(denoised_image, kernel)
    return denoised_image

# Define the objective function to be minimized (negative PSNR)
def objective_function(params):
    kernel_flat, lambda_value = params[:-1], params[-1]
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = denoise_image(noisy_image, kernel, lambda_value)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Generate a random initial set of parameters (kernel and lambda)
np.random.seed(42)  # Set a random seed for reproducibility
initial_params = np.random.rand(np.prod(kernel_size) + 1)

# Reshape the initial kernel and lambda to a flat array for optimization
initial_params_flat = initial_params.flatten()

# Minimize the negative objective function to find the optimal parameters using Adam optimizer
result = minimize(objective_function, initial_params_flat, method='L-BFGS-B', options={'disp': True})

# Retrieve the optimal parameters
optimal_params_flat = result.x
optimal_kernel_flat, optimal_lambda = optimal_params_flat[:-1], optimal_params_flat[-1]
optimal_kernel = optimal_kernel_flat.reshape(kernel_size)

# Calculate the denoised image using the optimal parameters
denoised_image = denoise_image(noisy_image, optimal_kernel, optimal_lambda)

# Calculate the PSNR value using the denoised image
optimal_psnr = compute_psnr(clean_image, denoised_image)

# Print the optimal kernel, lambda, and PSNR value
print("Optimal Kernel:")
print(optimal_kernel)
print("Optimal Lambda:", optimal_lambda)
print("Optimal PSNR value:", optimal_psnr)

Optimal Kernel:
[[0.37454012 0.95071431 0.73199394]
 [0.59865848 0.15601864 0.15599452]
 [0.05808361 0.86617615 0.60111501]]
Optimal Lambda: 0.7080725777960455
Optimal PSNR value: 4.689506522902148


### Testing Everything

In [15]:
import numpy as np
import cv2
from scipy.optimize import minimize

# Define the parameters
dt = 0.05  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image using a given kernel
def denoise_image(image, kernel, lambda_value):
    # Define the hyperparameter lambda
    def g(s_squared):
        return 1 / (1 + (s_squared + lambda_value**2))
    
    # Function to apply the update rule
    def update_rule(image, kernel):
        k_u = cv2.filter2D(image, -1, kernel)
        phi = k_u * g(k_u**2)
        p = dt * phi
        return image + p
    
    denoised_image = image.copy()
    for _ in range(50):
        denoised_image = update_rule(denoised_image, kernel)
    return denoised_image

# Define the objective function to be minimized (negative PSNR)
def objective_function(params):
    kernel_flat, lambda_value = params[:-1], params[-1]
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = denoise_image(noisy_image, kernel, lambda_value)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Generate a random initial set of parameters (kernel and lambda)
np.random.seed(42)  # Set a random seed for reproducibility
initial_params = np.random.rand(np.prod(kernel_size) + 1)

# Reshape the initial kernel and lambda to a flat array for optimization
initial_params_flat = initial_params.flatten()

# Define the optimization methods to try
optimization_methods = [
    'Nelder-Mead',
    'Powell',
    'CG',
    'BFGS',
    'L-BFGS-B',
    'TNC',
    'COBYLA',
    'SLSQP',
    'trust-constr'
]

# Define the hyperparameter values to try
lambda_values = [0.1, 0.01, 0.001]

# Define the best values
best_psnr = -np.inf
best_kernel = None
best_lambda = None
best_method = None

# Loop over optimization methods
for method in optimization_methods:
    # Loop over lambda values
    for lambda_value in lambda_values:
        # Minimize the negative objective function to find the optimal parameters using the current method
        result = minimize(objective_function, initial_params_flat, method=method)
        
        # Retrieve the optimal parameters
        optimal_params_flat = result.x
        optimal_kernel_flat, optimal_lambda = optimal_params_flat[:-1], optimal_params_flat[-1]
        optimal_kernel = optimal_kernel_flat.reshape(kernel_size)
        
        # Calculate the denoised image using the optimal parameters
        denoised_image = denoise_image(noisy_image, optimal_kernel, optimal_lambda)
        
        # Calculate the PSNR value using the denoised image
        optimal_psnr = compute_psnr(clean_image, denoised_image)
        
        # Check if this combination yields a higher PSNR value
        if optimal_psnr > best_psnr:
            best_psnr = optimal_psnr
            best_kernel = optimal_kernel
            best_lambda = optimal_lambda
            best_method = method

# Print the best combination of kernel, lambda, and optimization method
print("Best Optimization Method:", best_method)
print("Best Kernel:")
print(best_kernel)
print("Best Lambda:", best_lambda)
print("Best PSNR value:", best_psnr)

Best Optimization Method: COBYLA
Best Kernel:
[[ 32.65276265  62.44424515 -13.26629468]
 [ 74.12108044 193.25181469  28.85619094]
 [ 27.37130924  61.0555358  -15.59248367]]
Best Lambda: -0.7185035800521813
Best PSNR value: 14.006737745123843


#### Above but with grid search for lambda and different size kernels

In [19]:
import numpy as np
import cv2
from scipy.optimize import minimize

# Define the parameters
dt = 0.05  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image using a given kernel
def denoise_image(image, kernel, lambda_value):
    # Define the hyperparameter lambda
    def g(s_squared):
        return 1 / (1 + (s_squared + lambda_value**2))
    
    # Function to apply the update rule
    def update_rule(image, kernel):
        k_u = cv2.filter2D(image, -1, kernel)
        phi = k_u * g(k_u**2)
        p = dt * phi
        return image + p
    
    denoised_image = image.copy()
    for _ in range(50):
        denoised_image = update_rule(denoised_image, kernel)
    return denoised_image

# Define the objective function to be minimized (negative PSNR)
def objective_function(params):
    kernel_flat, lambda_value = params[:-1], params[-1]
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = denoise_image(noisy_image, kernel, lambda_value)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Generate a random initial set of parameters (kernel and lambda)
np.random.seed(42)  # Set a random seed for reproducibility
initial_params = np.random.rand(np.prod(kernel_size) + 1)

# Reshape the initial kernel and lambda to a flat array for optimization
initial_params_flat = initial_params.flatten()

# Define the optimization methods to try
optimization_methods = [
    'Nelder-Mead',
    'Powell',
    'CG',
    'BFGS',
    'L-BFGS-B',
    'TNC',
    'COBYLA',
    'SLSQP',
    'trust-constr'
]

# Define the hyperparameter values to try
lambda_values = np.logspace(-4, -1, num=10)

# Define the best values
best_psnr = -np.inf
best_kernel = None
best_lambda = None
best_method = None

# Loop over optimization methods
for method in optimization_methods:
    # Loop over lambda values
    for lambda_value in lambda_values:
        # Minimize the negative objective function to find the optimal parameters using the current method
        result = minimize(objective_function, initial_params_flat, method=method)
        
        # Retrieve the optimal parameters
        optimal_params_flat = result.x
        optimal_kernel_flat, optimal_lambda = optimal_params_flat[:-1], optimal_params_flat[-1]
        optimal_kernel = optimal_kernel_flat.reshape(kernel_size)
        
        # Calculate the denoised image using the optimal parameters
        denoised_image = denoise_image(noisy_image, optimal_kernel, optimal_lambda)
        
        # Calculate the PSNR value using the denoised image
        optimal_psnr = compute_psnr(clean_image, denoised_image)
        
        # Check if this combination yields a higher PSNR value
        if optimal_psnr > best_psnr:
            best_psnr = optimal_psnr
            best_kernel = optimal_kernel
            best_lambda = optimal_lambda
            best_method = method

# Print the best combination of kernel, lambda, and optimization method
print("Best Optimization Method:", best_method)
print("Best Kernel Size:", best_kernel.shape)
print("Best Kernel:")
print(best_kernel)
print("Best Lambda:", best_lambda)
print("Best PSNR value:", best_psnr)


Best Optimization Method: COBYLA
Best Kernel Size: (3, 3)
Best Kernel:
[[ 41.2427651   85.79857538  -8.00967482]
 [ 93.2627195  290.67955198  30.58112486]
 [ 23.83903915  72.34119016 -24.11976401]]
Best Lambda: -0.8648198437744031
Best PSNR value: 14.013415274911573


#### Non random initialization

In [1]:
import numpy as np
import cv2
from scipy.optimize import minimize, differential_evolution, shgo

# Define the parameters
dt = 0.05  # Time step
kernel_size = (3, 3)  # Size of the kernel

# Load the images
noisy_image = cv2.imread('noisy_cameraman.jpg', 0)
noisy_image = noisy_image.astype(np.float32) / 255.0

clean_image = cv2.imread('cameraman.jpg', 0)
clean_image = clean_image.astype(np.float32) / 255.0

# Function to compute the PSNR between two images
def compute_psnr(original, denoised):
    mse = np.mean((original - denoised) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse))
    return psnr

# Function to compute the denoised image using a given kernel
def denoise_image(image, kernel, lambda_value):
    # Define the hyperparameter lambda
    def g(s_squared):
        return 1 / (1 + (s_squared + lambda_value**2))
    
    # Function to apply the update rule
    def update_rule(image, kernel):
        k_u = cv2.filter2D(image, -1, kernel)
        phi = k_u * g(k_u**2)
        p = dt * phi
        return image + p
    
    denoised_image = image.copy()
    for _ in range(50):
        denoised_image = update_rule(denoised_image, kernel)
    return denoised_image

# Define the objective function to be minimized (negative PSNR)
def objective_function(params):
    kernel_flat, lambda_value = params[:-1], params[-1]
    kernel = kernel_flat.reshape(kernel_size)
    denoised_image = denoise_image(noisy_image, kernel, lambda_value)
    psnr = compute_psnr(clean_image, denoised_image)
    return -psnr

# Generate a random initial set of parameters (kernel and lambda)
np.random.seed(42)  # Set a random seed for reproducibility
initial_params = np.random.rand(np.prod(kernel_size) + 1)

# Set the initial kernel as [[0,1,0],[1,-4,1],[0,1,0]]
initial_kernel = np.array([[0,1,0],[1,-4,1],[0,1,0]])
initial_params[:-1] = initial_kernel.flatten()

# Reshape the initial kernel and lambda to a flat array for optimization
initial_params_flat = initial_params.flatten()

# Define the optimization methods to try
optimization_methods = [
    'Nelder-Mead',
    'Powell',
    'CG',
    'BFGS',
    'L-BFGS-B',
    'TNC',
    'COBYLA',
    'SLSQP',
    'trust-constr',
    'differential_evolution',
    'shgo'
]

# Define the hyperparameter values to try
lambda_values = lambda_values = np.linspace(0.001, 0.1, num=50)

# Define the best values
best_psnr = -np.inf
best_kernel = None
best_lambda = None
best_method = None

# Loop over optimization methods
for method in optimization_methods:
    # Loop over lambda values
    for lambda_value in lambda_values:
        # Minimize the negative objective function to find the optimal parameters using the current method
        if method == 'differential_evolution':
            result = differential_evolution(objective_function, bounds=[(0, 1)] * np.prod(kernel_size) + [(0, 1)],
                                            strategy='best1bin', popsize=10, tol=1e-6, mutation=(0.5, 1),
                                            recombination=0.7, seed=42)
        elif method == 'shgo':
            result = shgo(objective_function, bounds=[(0, 1)] * np.prod(kernel_size) + [(0, 1)])
        
        else:
            result = minimize(objective_function, initial_params_flat, method=method)
        
        # Retrieve the optimal parameters
        optimal_params_flat = result.x
        optimal_kernel_flat, optimal_lambda = optimal_params_flat[:-1], optimal_params_flat[-1]
        optimal_kernel = optimal_kernel_flat.reshape(kernel_size)
        
        # Calculate the denoised image using the optimal parameters
        denoised_image = denoise_image(noisy_image, optimal_kernel, optimal_lambda)
        
        # Calculate the PSNR value using the denoised image
        optimal_psnr = compute_psnr(clean_image, denoised_image)
        
        # Check if this combination yields a higher PSNR value
        if optimal_psnr > best_psnr:
            best_psnr = optimal_psnr
            best_kernel = optimal_kernel
            best_lambda = optimal_lambda
            best_method = method

# Print the best combination of kernel, lambda, and optimization method
print("Best Optimization Method:", best_method)
print("Best Kernel Size:", best_kernel.shape)
print("Best Kernel:")
print(best_kernel)
print("Best Lambda:", best_lambda)
print("Best PSNR value:", best_psnr)

Best Optimization Method: Nelder-Mead
Best Kernel Size: (3, 3)
Best Kernel:
[[ 3.19467146e-03  6.62185876e-01  2.41337026e-03]
 [-2.51111441e-01 -2.80013914e+00  1.65719812e+00]
 [ 1.46204454e-03  7.23178109e-01  2.50254011e-03]]
Best Lambda: 0.002169403504615088
Best PSNR value: 21.232611227089535
